# Background

This noteboek belongs to a series of notebooks for the research paper "Text Generation: Story Ending Prediction
Could T5 model understand the causal relationship and generate reasonable story endings?". 

The series of notebook include:
* SimpleT5_5outputs.py
* Model.ipynb
* Inference.ipynb
* Error Analysis.ipynb

This is the Inference notebook, and it can be ran in Google Colab or local computers.


# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Sun Nov 21 17:53:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load Packages

In [ ]:

!pip install torch
!pip install sklearn
!pip install transformers
!pip install rich[jupyter]
!pip install pytorch_lightning
!pip install SentencePiece
# !pip install --upgrade simplet5


     |████████████████████████████████| 3.1 MB 13.3 MB/s 
     |████████████████████████████████| 895 kB 33.8 MB/s 
     |████████████████████████████████| 596 kB 51.0 MB/s 
     |████████████████████████████████| 59 kB 7.3 MB/s 
     |████████████████████████████████| 3.3 MB 41.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 213 kB 12.7 MB/s 
     |████████████████████████████████| 51 kB 8.1 MB/s 
     |████████████████████████████████| 1.0 MB 12.3 MB/s 
     |████████████████████████████████| 132 kB 51.3 MB/s 
     |████████████████████████████████| 329 kB 46.2 MB/s 
     |████████████████████████████████| 829 kB 39.4 MB/s 
     |████████████████████████████████| 1.1 MB 36.7 MB/s 
     |████████████████████████████████| 192 kB 52.9 MB/s 
     |████████████████████████████████| 160 kB 50.2 MB/s 
     |████████████████████████████████| 271

# Load Model

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import sys  
sys.path.insert(0, '/content/drive/MyDrive/Colab Notebooks')
import simplet5_5output
from simplet5_5output import SimpleT5
import torch
import torch.nn.functional as F
from transformers import T5Tokenizer, T5ForConditionalGeneration

from sklearn.metrics import f1_score


Global seed set to 42


UPDATE 5 output


In [ ]:
from torch import cuda

device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
model= SimpleT5()
model_path = '/content/drive/MyDrive/ROCstory/output/full_output5_epoch8.zip (Unzipped Files)/full_output5_epoch8'
model.load_model("t5", model_path, use_gpu= True)

# Inference the val set

In [ ]:
val = pd.read_csv('/content/drive/MyDrive/ROCstory/data/cloze_test_val__winter2018-cloze_test_ALL_val.csv')
test = pd.read_csv('/content/drive/MyDrive/ROCstory/data/cloze_test_val__winter2018-cloze_test_ALL_test.csv')

In [ ]:
val_output = []
val_cols = ['InputSentence1', 'InputSentence2', 'InputSentence3', 'InputSentence4']
val['combined'] = val[val_cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

for index, row in val.iterrows():
  prediction = model.predict(row['combined'])
  val_output.append(prediction)

val['output']= val_output

val_5 = val.explode('output')

In [ ]:
val_5.head()

,InputStoryid,InputSentence1,InputSentence2,InputSentence3,InputSentence4,RandomFifthSentenceQuiz1,RandomFifthSentenceQuiz2,AnswerRightEnding,combined,output
0,138d5bfb-05cc-41e3-bf2c-fa85ebad14e2,Rick grew up in a troubled household.,"He never found good support in family, and tur...",It wasn't long before Rick got shot in a robbery.,The incident caused him to turn a new leaf.,He is happy now.,He joined a gang.,1,Rick grew up in a troubled household. He never...,He started his own gang.
0,138d5bfb-05cc-41e3-bf2c-fa85ebad14e2,Rick grew up in a troubled household.,"He never found good support in family, and tur...",It wasn't long before Rick got shot in a robbery.,The incident caused him to turn a new leaf.,He is happy now.,He joined a gang.,1,Rick grew up in a troubled household. He never...,"Now, Rick lives in a homeless shelter."
0,138d5bfb-05cc-41e3-bf2c-fa85ebad14e2,Rick grew up in a troubled household.,"He never found good support in family, and tur...",It wasn't long before Rick got shot in a robbery.,The incident caused him to turn a new leaf.,He is happy now.,He joined a gang.,1,Rick grew up in a troubled household. He never...,"He started his own gang, and became a drug add..."
0,138d5bfb-05cc-41e3-bf2c-fa85ebad14e2,Rick grew up in a troubled household.,"He never found good support in family, and tur...",It wasn't long before Rick got shot in a robbery.,The incident caused him to turn a new leaf.,He is happy now.,He joined a gang.,1,Rick grew up in a troubled household. He never...,"He started his own gang, and became a hermit."
0,138d5bfb-05cc-41e3-bf2c-fa85ebad14e2,Rick grew up in a troubled household.,"He never found good support in family, and tur...",It wasn't long before Rick got shot in a robbery.,The incident caused him to turn a new leaf.,He is happy now.,He joined a gang.,1,Rick grew up in a troubled household. He never...,"He started his own gang, and became a drug dea..."


# T5 similiarity score

In [ ]:
# apply T5 similarity score
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")
def T5_similarity (y, O):
  input_ids = tokenizer('stsb sentence1: ' + y + ' sentence2: ' + O, return_tensors = 'pt').input_ids
  outputs = model.generate(input_ids)
  # print(tokenizer.decode(outputs[0], skip_special_tokens=True))
  return tokenizer.decode(outputs[0], skip_special_tokens=True)


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [ ]:
def T5_max (y, O1, O2):
  score1 = T5_similarity(y, O1)
  score2 = T5_similarity(y, O2)

  choice = 1 if score1 > score2 else 2
  return choice

In [ ]:
val_5['T5_SS1'] = val_5.apply(lambda x: T5_similarity(x['output'],x['RandomFifthSentenceQuiz1']), axis=1)
val_5['T5_SS2'] = val_5.apply(lambda x: T5_similarity(x['output'],x['RandomFifthSentenceQuiz2']), axis=1)
val_5['T5_result'] = val_5.apply(lambda x: T5_max(x['output'],x['RandomFifthSentenceQuiz1'], x['RandomFifthSentenceQuiz2']), axis=1)

In [ ]:
val_t5_result= val_5.groupby(['combined', 'T5_result']).size().to_frame('t5_count').reset_index()

In [ ]:
idx = val_t5_result.groupby(['combined'])['t5_count'].transform(max) == val_t5_result['t5_count']
val_t5_result = val_t5_result[idx]

In [ ]:
val = val.merge(val_t5_result, on = 'combined', how = 'left')

In [ ]:
val_acc_t5 = np.sum(val['T5_result'] == val['AnswerRightEnding'])/len(val['T5_result'])
val_f1_t5 = f1_score(val['AnswerRightEnding'], val['T5_result'])

# Universal Sentence Encoder

In [ ]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.
INFO:absl:Downloaded https://tfhub.dev/google/universal-sentence-encoder/4, Total size: 987.47MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [ ]:
def plot_similarity(labels, features, rotation):
  corr = np.inner(features, features)
  print(corr)
  sns.set(font_scale=1.2)
  g = sns.heatmap(
      corr,
      xticklabels=labels,
      yticklabels=labels,
      vmin=0,
      vmax=1,
      cmap="YlOrRd")
  g.set_xticklabels(labels, rotation=rotation)
  g.set_title("Semantic Textual Similarity")

def run_and_plot(messages_):
  message_embeddings_ = embed(messages_)
  plot_similarity(messages_, message_embeddings_, 90)

def USE_similarity(messages_):
  message_embeddings_ = embed(messages_)
  corr = np.inner( message_embeddings_, message_embeddings_)
  score1 = corr[0][1]
  score2 = corr[0][2]
  choice = 1 if score1 > score2 else 2

  return (score1, score2, choice)

In [ ]:
val_5['messages'] = val_5.apply(lambda x: list([x['output'],
                                        x['RandomFifthSentenceQuiz1'],
                                        x['RandomFifthSentenceQuiz2']]),axis=1) 
val_5['USE_similarity'] = val_5.apply(lambda x: USE_similarity(x['messages']), axis=1)


val_5.loc[:,'SS1_USE']=val_5.USE_similarity.map(lambda x:x[0])
val_5.loc[:,'SS2_USE']=val_5.USE_similarity.map(lambda x:x[1])
val_5.loc[:,'result_USE']=val_5.USE_similarity.map(lambda x:x[2])


In [ ]:
val_use_result= val_5.groupby(['combined', 'result_USE']).size().to_frame('use_count').reset_index()

In [ ]:
idx = val_use_result.groupby(['combined'])['use_count'].transform(max) == val_use_result['use_count']
val_use_result = val_use_result[idx]

In [ ]:
val = val.merge(val_use_result, on = 'combined', how = 'left')

In [ ]:
val_acc_USE = np.sum(val['result_USE'] == val['AnswerRightEnding'])/len(val['result_USE'])
val_f1_USE = f1_score(val['AnswerRightEnding'], val['result_USE'])

In [ ]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()

# put in dataframe
pd.DataFrame([[now, val_acc_t5,val_f1_t5, val_acc_USE, val_f1_USE]],
             columns=['Data','T5 Acc','T5 F1', 'USE Acc', 'USE F1' ])

,Data,T5 Acc,T5 F1,Cosine Similarity Acc,Cosine Similarity F1,USE Acc,USE F1
0,2021-11-21 21:16:17.410840,0.721833,0.684932,0.587524,0.575916,0.674729,0.683983


# Simi-Senti Scores

In [ ]:
!pip install flair

In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence

In [ ]:
classifier = TextClassifier.load('en-sentiment')

In [ ]:
def get_label (input):
  sentences = result.apply(lambda x: Sentence(x[input]), axis=1)
  sentences.apply(lambda x:classifier.predict(x))
  labels = sentences.apply(lambda x: x.labels)
  labels = labels.apply(lambda x: str(x[0]).split()[0])
  return labels

In [ ]:
val_5['output_sentiment'] = get_label('output')
val_5['ending1_sentiment'] = get_label('RandomFifthSentenceQuiz1')
val_5['ending2_sentiment'] = get_label('RandomFifthSentenceQuiz2')

In [ ]:
val_5['ending1_sentiment_result'] = (val_5['ending1_sentiment'] == val_5['output_sentiment']).astype(int) 
val_5['ending2_sentiment_result'] = (val_5['ending2_sentiment'] == val_5['output_sentiment']).astype(int)

val_5['T5_sentiment1'] = val_5['T5_SS1'] + 1* val_5['ending1_sentiment_result']
val_5['T5_sentiment2'] = val_5['T5_SS2'] + 1* val_5['ending2_sentiment_result']

In [ ]:
from random import randint, choice

import random
def t5_max(s1, s2, a1, a2):
  if s1 > s2:
    return 1
  elif s1 < s2:
    return 2
  else:
    if a1 > a2:
      return 1
    elif a1 < a2:
      return 2
val_5['T5_senti_result'] = val_5.apply(lambda x: t5_max(x['T5_sentiment1'], x['T5_sentiment2'], x['SS1_USE'], x['SS2_USE']), axis = 1)

In [ ]:
t5_senti_result= val_5.groupby(['combined', 'T5_senti_result']).size().to_frame('t5_count').reset_index()
idx = t5_senti_result.groupby(['combined'])['t5_count'].transform(max) == t5_senti_result['t5_count']
t5_senti_result = t5_senti_result[idx]

In [ ]:
val_5 = val_5.merge(t5_senti_result, on = 'combined', how = 'left')

In [ ]:
# output val_5 for further analysis
val_5.to_csv( '/content/drive/MyDrive/ROCstory/output/epoch8_val_5output.csv')